In [ ]:
import os
import time
import json
import numpy as np
import torch
import joblib
from pathlib import Path
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from data.trajectory_loader import load_dataset
from models.attention_bi_gru_predictor import TrajPredictor
from utils.logger import get_logger, log_metrics_for_features
from utils.plot_generator import plot_multiagent_trajectories, plot_3d_trajectories_subplots
from utils.scaler import scale_per_agent

In [ ]:
# pylint: disable=all
# Data settings and parameters
DATA_TYPE = "simulated"  # Options: "zurich", "quadcopter", "mixed", "simulated"
SEQUENTIAL_PREDICTION = (
    True  # If False, model predicts only the last point for FORWARD_LEN steps
)
EMBEDDING_EXTRACTION = True
AGENTS = 6  # Number of agents or drones
LOOK_BACK = 50  # Number of past time steps to use as input
if EMBEDDING_EXTRACTION:
    FORWARD_LEN = 50 # Prediction is the same as LOOK_BACK for embedding extraction
else:
    FORWARD_LEN = 5  # Number of future time steps to predict
FEATURES_PER_AGENT = 6  # x, y, z, vx, vy, vz, ax, ay, az

# Training parameters
BATCH_SIZE = 32
EPOCHS = 500
LEARNING_RATE = 1e-3

# Plotting parameters
NUM_PLOTS = 2  # number of plots to generate
NUM_SUBPLOTS = 2  # number of subplots to generate

In [3]:
# Setup logger and experiment folder
logger, exp_dir = get_logger()
os.makedirs(exp_dir, exist_ok=True)

logger.info("Experiment started")
logger.info("Experiment folder: %s", exp_dir)
logger.info("Dataset: %s", DATA_TYPE)
logger.info("Number of drones (agents): %d", AGENTS)
logger.info("LOOK_BACK (past steps): %d", LOOK_BACK)
logger.info("FORWARD_LEN (future steps): %d", FORWARD_LEN)
logger.info("SEQUENTIAL_PREDICTION: %s", "True" if SEQUENTIAL_PREDICTION else "False")

2025-11-04 17:13:01,207 - INFO - Experiment started
2025-11-04 17:13:01,208 - INFO - Experiment folder: experiments/20251104_171301
2025-11-04 17:13:01,209 - INFO - Dataset: simulated
2025-11-04 17:13:01,210 - INFO - Number of drones (agents): 6
2025-11-04 17:13:01,211 - INFO - LOOK_BACK (past steps): 50
2025-11-04 17:13:01,212 - INFO - FORWARD_LEN (future steps): 5
2025-11-04 17:13:01,212 - INFO - SEQUENTIAL_PREDICTION: True


In [ ]:
# Load DataFrame
df = load_dataset(
    DATA_TYPE,
    min_rows=800,
    num_flights=AGENTS,
    features_per_agent=FEATURES_PER_AGENT,
)
print(df.shape)

# Prepare sequences
# Track trajectory indices to be used in plotting later
X, y, trajectory_ids = [], [], []

for traj_idx in df["trajectory_index"].unique():
    traj_df = df[df["trajectory_index"] == traj_idx].reset_index(drop=True)

    # Drop trajectory_index for features
    traj_data = traj_df.drop(columns=["trajectory_index"]).values.astype(np.float32)
    n_rows = traj_data.shape[0]

    seq_count = n_rows - LOOK_BACK - FORWARD_LEN + 1
    for i in range(seq_count):
        seq_X = traj_data[i : i + LOOK_BACK]  # shape (LOOK_BACK, features)

        # For sequential prediction, predict FORWARD_LEN steps; else just the last step
        if EMBEDDING_EXTRACTION is False:
            if SEQUENTIAL_PREDICTION:
                seq_y = traj_data[
                    i + LOOK_BACK : i + LOOK_BACK + FORWARD_LEN
                ]  # shape (FORWARD_LEN, features)
            else:
                seq_y = traj_data[
                    i + LOOK_BACK + FORWARD_LEN - 1 : i + LOOK_BACK + FORWARD_LEN
                ]  # (1, features)
        else:
            seq_y = seq_X.copy()  # For embedding extraction, target is same as input

        X.append(seq_X)
        y.append(seq_y)
        trajectory_ids.append(traj_idx)
        

(27894, 55)


In [5]:
# Convert to NumPy arrays
X = np.array(X, dtype=np.float32)  # (num_sequences, LOOK_BACK, features)
y = np.array(
    y, dtype=np.float32
)  # (num_sequences, 1, features) or (num_sequences, FORWARD_LEN, features)
trajectory_ids = np.array(trajectory_ids)

# Split train/test
X_train, X_test, y_train, y_test, traj_train, traj_test = train_test_split(
    X, y, trajectory_ids, test_size=0.2, shuffle=False
)

# Get feature dimension dynamically
num_features_X = X_train.shape[-1]

# Scale data to [0, 1]
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

# Fit scalers on training data only
X_train_scaled = scale_per_agent(X_train, scaler_X, FEATURES_PER_AGENT, fit=True)
X_test_scaled = scale_per_agent(X_test, scaler_X, FEATURES_PER_AGENT, fit=False)

y_train_scaled = scale_per_agent(y_train, scaler_y, FEATURES_PER_AGENT, fit=True)
y_test_scaled = scale_per_agent(y_test, scaler_y, FEATURES_PER_AGENT, fit=False)

# Save scalers
joblib.dump(scaler_X, os.path.join(exp_dir, "scaler_X.pkl"))
joblib.dump(scaler_y, os.path.join(exp_dir, "scaler_y.pkl"))

# Convert to tensors
X_train_tensor = torch.tensor(
    X_train_scaled, dtype=torch.float32
)  # (num_sequences, LOOK_BACK, features)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32)

# Create DataLoaders
train_loader = DataLoader(
    TensorDataset(X_train_tensor, y_train_tensor), batch_size=BATCH_SIZE, shuffle=True
)
test_loader = DataLoader(
    TensorDataset(X_test_tensor, y_test_tensor), batch_size=BATCH_SIZE, shuffle=False
)

In [6]:
# Log dataset sizes
total_sequences = X_train_tensor.shape[0] + X_test_tensor.shape[0]
logger.info("Total sequences: %d", total_sequences)
logger.info("Train sequences: %s", X_train_tensor.shape)
logger.info("Test sequences: %s", X_test_tensor.shape)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info("Using device: %s", device)

2025-11-04 17:13:02,056 - INFO - Total sequences: 27840
2025-11-04 17:13:02,058 - INFO - Train sequences: torch.Size([22272, 50, 54])
2025-11-04 17:13:02,059 - INFO - Test sequences: torch.Size([5568, 50, 54])
2025-11-04 17:13:02,059 - INFO - Using device: cpu


In [ ]:
# Model, criterion, optimizer
model_params = {
    "input_size": FEATURES_PER_AGENT,
    "enc_hidden_size": 64,
    "dec_hidden_size": 64,
    "num_layers": 1,
}
model = TrajPredictor(**model_params).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Log model info
logger.info("Model module: %s", model.__class__.__module__)
logger.info("Model architecture:\n%s", model)

2025-11-04 17:13:02,844 - INFO - Model module: models.attention_bi_gru_predictor
2025-11-04 17:13:02,844 - INFO - Model architecture:
TrajPredictor(
  (encoder): GRU(9, 128, batch_first=True, bidirectional=True)
  (attention): Attention(
    (attn): Linear(in_features=384, out_features=128, bias=True)
    (v): Linear(in_features=128, out_features=1, bias=False)
  )
  (enc_to_dec): Linear(in_features=256, out_features=128, bias=True)
  (decoder): GRU(265, 128, batch_first=True)
  (fc_out): Linear(in_features=128, out_features=9, bias=True)
)


In [ ]:
# Log training time
training_start = time.time()

# Early stopping parameters
patience = 10
best_loss = float("inf")
epochs_no_improve = 0
early_stop = False

# Training Loop
model.train()
try:
    for epoch in range(EPOCHS):
        total_loss = 0.0
        for batch_x, batch_y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            optimizer.zero_grad()

            # Add prediction_len argument if sequential prediction
            if SEQUENTIAL_PREDICTION:
                pred = model(batch_x, pred_len=FORWARD_LEN)
            else:
                pred = model(batch_x, pred_len=1)

            loss = criterion(pred, batch_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        logger.info("Epoch %d/%d - Train Loss: %.7f", epoch + 1, EPOCHS, avg_loss)

        # Early stopping
        if avg_loss < best_loss:
            best_loss = avg_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), os.path.join(exp_dir, "best_model.pt"))
        else:
            epochs_no_improve += 1

        # If no improvement for 'patience' epochs, stop training
        if epochs_no_improve >= patience:
            logger.info("Early stopping triggered after %d epochs", epoch + 1)
            early_stop = True
            break

except KeyboardInterrupt:
    logger.warning("Training interrupted by user! Running evaluation...")

# Save last-epoch model
finally:
    torch.save(model.state_dict(), os.path.join(exp_dir, "last_model.pt"))

# If training completed without early stopping
if not early_stop:
    logger.info("Training finished without early stopping.")

# Log total training time
training_end_time = time.time()
elapsed_time = training_end_time - training_start
logger.info("Total training time: %.2f seconds", elapsed_time)

2025-11-04 17:16:31,150 - INFO - Epoch 1/500 - Train Loss: 0.0023183
2025-11-04 17:19:58,825 - INFO - Epoch 2/500 - Train Loss: 0.0000705
2025-11-04 17:23:29,290 - INFO - Epoch 3/500 - Train Loss: 0.0000619
2025-11-04 17:26:57,360 - INFO - Epoch 4/500 - Train Loss: 0.0000587
2025-11-04 17:30:45,533 - INFO - Epoch 5/500 - Train Loss: 0.0000565
2025-11-04 17:34:37,781 - INFO - Epoch 6/500 - Train Loss: 0.0000577
2025-11-04 17:38:33,657 - INFO - Epoch 7/500 - Train Loss: 0.0000552
2025-11-04 17:42:28,006 - INFO - Epoch 8/500 - Train Loss: 0.0000553
2025-11-04 17:46:21,608 - INFO - Epoch 9/500 - Train Loss: 0.0000541
2025-11-04 17:50:27,207 - INFO - Epoch 10/500 - Train Loss: 0.0000536
2025-11-04 17:54:20,894 - INFO - Epoch 11/500 - Train Loss: 0.0000507
2025-11-04 17:58:14,437 - INFO - Epoch 12/500 - Train Loss: 0.0000509
2025-11-04 18:02:10,396 - INFO - Epoch 13/500 - Train Loss: 0.0000494
2025-11-04 18:06:07,591 - INFO - Epoch 14/500 - Train Loss: 0.0000493
2025-11-04 18:06:41,978 - WAR

In [ ]:
# Evaluation parameters
all_preds, all_trues = [], []
inference_times = []
test_loss = 0.0
total_sequences = 0

model.eval()
with torch.no_grad():
    for batch_x, batch_y in tqdm(test_loader, desc="Evaluating"):
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        total_sequences += batch_x.size(0)

        # Measure inference time
        start_time = time.time()

        # Add prediction_len argument if sequential prediction
        if SEQUENTIAL_PREDICTION:
            outputs = model(batch_x, pred_len=FORWARD_LEN)
        else:
            outputs = model(batch_x, pred_len=1)

        end_time = time.time()

        # Record inference time per batch
        inference_times.append(end_time - start_time)

        # Compute test loss
        loss = criterion(outputs, batch_y)
        test_loss += loss.item()

        # Store predictions and true values
        all_preds.append(outputs.cpu())
        all_trues.append(batch_y.cpu())

# Compute average test loss
avg_test_loss = test_loss / len(test_loader)

# Compute total inference time, average inference time per batch and per sequence
total_inf_time = sum(inference_times)
avg_inf_time_per_seq = total_inf_time / total_sequences
avg_inf_time_per_batch = total_inf_time / len(test_loader)

# Log final test metrics
logger.info("Test Loss (scaled): %.7f", avg_test_loss)
logger.info("Average inference time per sequence: %.6f seconds", avg_inf_time_per_seq)
logger.info("Average inference time per batch: %.6f seconds", avg_inf_time_per_batch)

2025-11-04 18:07:06,735 - INFO - Test Loss (scaled): 0.0000529
2025-11-04 18:07:06,736 - INFO - Average inference time per sequence: 0.004007 seconds
2025-11-04 18:07:06,736 - INFO - Average inference time per batch: 0.128212 seconds


In [ ]:
# Concatenate all batches
y_pred = torch.cat(all_preds, dim=0)
y_true = torch.cat(all_trues, dim=0)

log_metrics_for_features(
    y_true, y_pred, scaler_y, AGENTS, FEATURES_PER_AGENT, logger
)

2025-11-04 18:07:28,163 - INFO - ------------------------------------------------------------------------------------------------------------------------------------------
2025-11-04 18:07:28,164 - INFO - Timestep |        EDE |    Pos_MSE |   Pos_RMSE |    Pos_MAE |    Vel_MSE |   Vel_RMSE |    Vel_MAE |    Acc_MSE |   Acc_RMSE |    Acc_MAE
2025-11-04 18:07:28,164 - INFO - ------------------------------------------------------------------------------------------------------------------------------------------
2025-11-04 18:07:28,165 - INFO -        0 |   0.389369 |   0.178855 |   0.422913 |   0.176450 |   0.010790 |   0.103877 |   0.066555 |   3.696356 |   1.922591 |   0.802852
2025-11-04 18:07:28,165 - INFO -        1 |   0.365166 |   0.286486 |   0.535244 |   0.172274 |   0.015148 |   0.123077 |   0.073415 |   3.587385 |   1.894039 |   0.828413
2025-11-04 18:07:28,166 - INFO -        2 |   0.371952 |   0.403226 |   0.635001 |   0.178672 |   0.021284 |   0.145889 |   0.083912 |   3.5

In [11]:
# Save config / hyperparameters
config = {
    "device": str(device),
    "model_module": model.__class__.__module__,
    "model_class": model.__class__.__name__,
    "model_params": model_params,
    "DATA_TYPE": DATA_TYPE,
    "AGENTS": AGENTS,
    "FEATURES_PER_AGENT": FEATURES_PER_AGENT,
    "LOOK_BACK": LOOK_BACK,
    "FORWARD_LEN": FORWARD_LEN,
    "SEQUENTIAL_PREDICTION": SEQUENTIAL_PREDICTION,
    "EPOCHS": EPOCHS,
    "BATCH_SIZE": BATCH_SIZE,
    "LEARNING_RATE": LEARNING_RATE,
}

config_path = os.path.join(exp_dir, "config.json")
with open(config_path, "w", encoding="utf-8") as f:
    json.dump(config, f, indent=4)

logger.info("Config saved to %s", config_path)

2025-11-04 18:07:57,588 - INFO - Config saved to experiments/20251104_171301/config.json


In [ ]:
# Group results back by trajectory_index
traj_test = traj_test[: len(y_true)]  # align just in case
unique_trajs = np.unique(traj_test)

# Randomly select trajectories to plot
plot_trajs = np.random.choice(
    unique_trajs, size=min(NUM_PLOTS, len(unique_trajs)), replace=False
).tolist()

# Plot trajectories using the helper function
plot_multiagent_trajectories(
    y_true=y_true.numpy(),
    y_pred=y_pred.numpy(),
    traj_ids=traj_test,
    plot_trajs=plot_trajs,
    scaler=scaler_y,
    features_per_agent=FEATURES_PER_AGENT,
    save_dir=exp_dir,
    velocity_scale=0.5,
    acceleration_scale=0.3
)

In [ ]:
# Stack all past inputs (for context)
past_inputs = torch.cat(
    [b for b, _ in test_loader], dim=0
).numpy()  # (num_sequences, LOOK_BACK, num_features)

# Ensure NUM_PLOTS does not exceed number of sequences
num_sequences = y_true.shape[0]
NUM_SUBPLOTS = min(NUM_SUBPLOTS, num_sequences)

# Randomly select sequence indices for plotting
plot_indices = np.random.choice(num_sequences, size=NUM_SUBPLOTS, replace=False)

trajectory_sets = []

for seq_idx in plot_indices:
    past = past_inputs[seq_idx]  # shape: (LOOK_BACK, features)
    true_future = y_true[seq_idx]  # shape: (seq_len, features)
    pred_future = y_pred[seq_idx]  # shape: (seq_len, features)

    # Inverse scale past and future
    past_orig = scale_per_agent(past, scaler_X, FEATURES_PER_AGENT, inverse=True)
    true_future_orig = scale_per_agent(
        true_future, scaler_y, FEATURES_PER_AGENT, inverse=True
    )
    pred_future_orig = scale_per_agent(
        pred_future, scaler_y, FEATURES_PER_AGENT, inverse=True
    )

    # Concatenate last past point with future to make continuous lines
    true_line = np.vstack([past_orig[-1:], true_future_orig])
    pred_line = np.vstack([past_orig[-1:], pred_future_orig])

    trajectory_sets.append((past_orig, true_line, pred_line))

# Create subplots for selected sequences
plot_path = Path(exp_dir) / "training_subplots.png"
plot_3d_trajectories_subplots(
    trajectory_sets,
    per_agent=True,
    num_features=FEATURES_PER_AGENT,            # Enable velocity + acceleration
    velocity_scale=0.5,        # Adjust arrow length
    acceleration_scale=0.3,    # Adjust arrow length
    save_path=str(plot_path)
)